In [1]:
!pip install langchain cohere python-dotenv gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 997.8/997.8 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.8/207.8 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 59.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 60.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 389.8/389.8 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.4/140.4 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
!pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 24.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 3.6 MB/s eta 0:00:00


In [9]:
import os
import gradio as gr
from dotenv import load_dotenv
from langchain.agents import initialize_agent
from langchain.agents import Tool
from langchain.agents import AgentType
from langchain.llms import Cohere
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.memory import ConversationBufferMemory

In [5]:
os.environ['COHERE_API_KEY'] = "nhYrFwn0xNFthWrVy6YwUEGES2hMwsc0hRDjTnlh"
os.environ['TAVILY_API_KEY'] = "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"

In [6]:
# Initialize API wrappers
search = TavilySearchResults()

In [10]:
# Define tools
tools = [
    Tool(
        name="Search",
        func=search.run,
        description="useful for when you need to answer questions about current events"
    )]

In [12]:
# Initialize the Cohere language model
llm = Cohere(temperature=0.4, model="command-r-plus")
#chat = ChatCohere(model="command-r-plus", temperature=0.3)

In [13]:
# Set up memory which allows for storing messages and then extracts the messages in a variable.
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

In [14]:
# Initialize the agent
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
    memory=memory,
    verbose=True
)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.3.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [15]:
def chatbot(message, history):
    # Convert history to the format expected by the agent
    chat_history = []
    for human, ai in history:
        chat_history.append(f"Human: {human}")
        chat_history.append(f"AI: {ai}")

    # Run the agent
    response = agent.run(input=message, chat_history=chat_history)

    # Return the response
    return response

In [16]:
iface = gr.ChatInterface(
    chatbot,
    chatbot=gr.Chatbot(height=300),
    textbox=gr.Textbox(placeholder="Ask me anything...", container=False, scale=7),
    title="AI Agent",
    description="Ask me anything, and I'll use my tools to find the answer!",
    theme="soft",
    examples=[
        "What is the difference between generative and discriminative models?",
        "Describe the architecture of a Generative Adversarial Network and how the generator and discriminator interact during training.",
        "Explain the concept of a Variational Autoencoder (VAE) and how it incorporates latent variables into its architecture.",
        "How do conditional generative models differ from unconditional ones? Provide an example scenario where a conditional approach is beneficial.",
        "What is mode collapse in the context of GANs, and what strategies can be employed to address it during training?",
        "How does overfitting manifest in generative models, and what techniques can be used to prevent it during training?",
        "What is gradient clipping, and how does it help in stabilizing the training process of generative models?",
        "what is langchain?",
        "what is rnn? explain the architecture of rnn?",
        "Discuss strategies for training generative models when the available dataset is limited?",
        "Explain how curriculum learning can be applied in the training of generative models. What advantages does it offer?",
        "Describe the concept of learning rate scheduling and its role in optimizing the training process of generative models over time?"
    ],
    cache_examples=True,
    retry_btn=None,
    undo_btn="Delete Previous",
    clear_btn="Clear",
)

Caching examples at: '/content/gradio_cached_examples/15'
Caching example 1/12


> Entering new AgentExecutor chain...


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


Thought: Do I need to use a tool? No
AI: Generative models and discriminative models are two different types of algorithms used in machine learning. 

Generative models aim to generate new data that follows the same statistical distribution as the training data. These models learn the underlying probability distribution of the data and can be used to generate new, synthetic data samples. Examples of generative models include Gaussian mixture models and generative adversarial networks (GANs).

On the other hand, discriminative models focus on learning a function that can discriminate between different classes or categories in the data. These models learn a mapping from input variables to output variables and are often used for classification or prediction tasks. Examples of discriminative models include logistic regression, decision trees, and neural networks.

In summary, the key difference lies in their objectives: generative models aim to generate new data, while discriminative model

In [ ]:
# Launch the interface
iface.launch(debug=True)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://e0822cb84b3d4c8410.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
